In [1]:
# ===== Cell 1: 載入必要套件 =====
import pandas as pd
import numpy as np
import glob
import os

# ===== Cell 2: 快速載入器 =====
class QuickLoader:
    """簡化版載入器，直接產生 arduino_features"""
    
    def __init__(self, data_dir='data/raw'):
        self.data_dir = data_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
        
        # R0 校正參考值
        self.R0_reference = {
            'MQ2_raw': 10.0,
            'MQ3_raw': 60.0,
            'MQ9_raw': 9.8,
            'MQ135_raw': 76.63,
            'TGS2602_raw': 2.5
        }
    
    def load_and_process(self):
        """一次性載入並處理到 arduino_features"""
        print("🔧 快速處理模式...")
        
        # 1. 載入所有檔案
        air_data, pineapple_data = self._load_files()
        
        if not pineapple_data:
            print("❌ 找不到鳳梨數據")
            return None
        
        # 2. 計算 Arduino 特徵
        arduino_features = self._calculate_features(air_data, pineapple_data)
        
        print(f"✅ 完成！產生 {len(arduino_features)} 顆鳳梨的特徵")
        return arduino_features
    
    def _load_files(self):
        """載入 Excel 檔案"""
        xlsx_files = glob.glob(os.path.join(self.data_dir, '*.xlsx'))
        print(f"📂 找到 {len(xlsx_files)} 個檔案")
        
        air_data = {}
        pineapple_data = {}
        
        for file_path in xlsx_files:
            filename = os.path.basename(file_path)
            parts = filename.replace('.xlsx', '').split('_')
            
            if len(parts) < 3:
                continue
            
            is_air = 'air' in filename
            
            # 讀取檔案
            try:
                # 嘗試跳過第一行
                df = pd.read_excel(file_path, engine='openpyxl', skiprows=1)
                
                # 檢查是否有正確的欄位
                if 'timestamp_ms' not in df.columns:
                    df = pd.read_excel(file_path, engine='openpyxl')
                
                # 轉換數字型別
                for col in self.sensor_cols + ['timestamp_ms']:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                
                # 清理缺值
                df = df.dropna().reset_index(drop=True)
                
            except Exception as e:
                print(f"⚠️  載入失敗 {filename}: {e}")
                continue
            
            # 分類儲存
            if is_air:
                if len(parts) == 3:
                    # 共用 air
                    date = parts[1]
                    air_data[f"shared_{date}"] = df
                else:
                    # 專屬 air
                    pid = parts[1]
                    date = parts[2]
                    air_data[f"{pid}_{date}"] = df
            else:
                pid = parts[1]
                date = parts[2]
                
                if pid not in pineapple_data:
                    pineapple_data[pid] = {}
                pineapple_data[pid][date] = df
        
        print(f"   ✓ Air: {len(air_data)}, Pineapple: {len(pineapple_data)} 顆")
        return air_data, pineapple_data
    
    def _calculate_features(self, air_data, pineapple_data):
        """計算 Arduino 特徵"""
        arduino_features = {}
        
        for pid, date_dict in pineapple_data.items():
            arduino_features[pid] = {}
            
            for date, pine_df in date_dict.items():
                # 找對應的 air baseline
                air_df = self._get_air(air_data, pid, date)
                
                if air_df is None:
                    print(f"⚠️  {pid}_{date} 找不到 air baseline")
                    continue
                
                # 計算 Delta
                min_len = min(len(pine_df), len(air_df))
                delta_df = pine_df.iloc[:min_len].copy()
                
                for col in self.sensor_cols:
                    if col in pine_df.columns and col in air_df.columns:
                        delta_df[f'{col}_delta'] = (
                            pine_df[col].iloc[:min_len].values - 
                            air_df[col].iloc[:min_len].values
                        )
                
                # 計算 Arduino 特徵
                features_df = delta_df.copy()
                
                for col in self.sensor_cols:
                    if col in delta_df.columns:
                        sensor_name = col.replace('_raw', '')
                        R0 = self.R0_reference.get(col, 10.0)
                        
                        # Rs/R0
                        features_df[f'{sensor_name}_Rs_R0'] = delta_df[col] / R0
                        
                        # Delta Rs/R0
                        delta_col = f'{col}_delta'
                        if delta_col in delta_df.columns:
                            features_df[f'{sensor_name}_delta_Rs_R0'] = delta_df[delta_col] / R0
                        
                        # 移動平均
                        features_df[f'{sensor_name}_ma10'] = (
                            delta_df[col].rolling(window=10, center=True).mean()
                        )
                        
                        # 移動標準差
                        features_df[f'{sensor_name}_std10'] = (
                            delta_df[col].rolling(window=10, center=True).std()
                        )
                
                arduino_features[pid][date] = features_df
                print(f"   ✓ {pid}_{date}: {features_df.shape}")
        
        return arduino_features
    
    def _get_air(self, air_data, pid, date):
        """取得對應的 air baseline"""
        # 專屬 air
        key = f"{pid}_{date}"
        if key in air_data:
            return air_data[key]
        
        # 共用 air
        shared_key = f"shared_{date}"
        if shared_key in air_data:
            return air_data[shared_key]
        
        return None

# 執行載入
loader = QuickLoader('data/raw')
arduino_features = loader.load_and_process()


🔧 快速處理模式...
📂 找到 164 個檔案


KeyboardInterrupt: 

In [ ]:
# ===== 載入 Step3 存好的成熟度標籤（4 等級）=====
import pickle
import os

pkl_path = 'data/processed/maturity_levels_4class.pkl'

if not os.path.exists(pkl_path):
    raise FileNotFoundError(
        f"找不到 {pkl_path}，請先在 step3.ipynb 跑完「重新映射階段」那一格，"
        "讓它產生 maturity_levels_4class.pkl"
    )

with open(pkl_path, 'rb') as f:
    maturity_levels = pickle.load(f)

print("✅ 已載入 maturity_levels_4class.pkl ，共有",
      len(maturity_levels), "顆鳳梨的成熟度標籤")


✅ 已載入 maturity_levels_4class.pkl ，共有 11 顆鳳梨的成熟度標籤


In [ ]:
# ===== Cell 11: Step 4 - 特徵工程（修正版）=====

import numpy as np
import pandas as pd
from scipy.stats import linregress
import pickle
import os

class FeatureEngineering:
    """
    使用固定時間窗口提取特徵
    目標：將時間序列轉換為 (n_samples, n_features) 的特徵矩陣
    """
    
    def __init__(self, arduino_features, maturity_labels, window_size=120):
        """
        參數:
            window_size: 時間窗口大小（秒）
                        假設取樣率 1Hz，120秒 = 120 個樣本點
        """
        self.arduino_features = arduino_features
        self.maturity_labels = maturity_labels
        self.window_size = window_size
        self.sensor_cols = ['MQ2', 'MQ3', 'MQ9', 'MQ135', 'TGS2602']
        
        self.feature_matrix = None
        self.labels = None
        self.metadata = None
        
        self.output_dir = 'data/processed'
        os.makedirs(self.output_dir, exist_ok=True)
    
    def extract_all_features(self):
        """批次提取所有特徵"""
        print("="*60)
        print("🔧 Step 4: 特徵工程（固定時間窗）")
        print("="*60)
        print(f"窗口大小: {self.window_size} 秒\n")
        
        all_features = []
        all_labels = []
        all_metadata = []
        
        for pid in self.arduino_features.keys():
            if pid not in self.maturity_labels:
                print(f"⚠️  {pid}: 找不到對應的成熟度標籤，跳過")
                continue
            
            print(f"🍍 處理 Pineapple {pid}...")
            
            # 合併該鳳梨的所有日期數據（這裡會處理「長度不一致」的狀況）
            combined_df, combined_labels = self._combine_pineapple_data(pid)
            
            if combined_df is None or len(combined_df) == 0:
                print(f"   ⚠️  {pid}: 沒有可用的樣本，跳過\n")
                continue
            
            # 滑動窗口提取特徵
            features, labels, metadata = self._sliding_window_extraction(
                combined_df, combined_labels, pid
            )
            
            if features.size == 0:
                print(f"   ⚠️  {pid}: 無法產生任何窗口，跳過\n")
                continue
            
            all_features.append(features)
            all_labels.append(labels)
            all_metadata.extend(metadata)
            
            print(f"   ✓ 提取 {len(features)} 個窗口，每個 {features.shape[1]} 維特徵\n")
        
        # 合併所有鳳梨的特徵
        self.feature_matrix = pd.DataFrame(
            np.vstack(all_features),
            columns=self._get_feature_names()
        )
        self.labels = np.hstack(all_labels)
        self.metadata = pd.DataFrame(all_metadata)
        
        print("="*60)
        print(f"✅ 特徵提取完成！")
        print(f"   特徵矩陣: {self.feature_matrix.shape}")
        print(f"   標籤數量: {len(self.labels)}")
        print(f"   標籤分布: {dict(zip(*np.unique(self.labels, return_counts=True)))}")
        print("="*60)
        
        return self.feature_matrix, self.labels, self.metadata
    
    def _combine_pineapple_data(self, pid):
        """合併單顆鳳梨的所有日期數據
           並處理「資料長度 > 標籤長度」的狀況
        """
        df_list = []
        label_list = []
        
        date_dict = self.arduino_features[pid]
        labels = self.maturity_labels[pid]
        total_label_len = len(labels)
        
        offset = 0
        for date in sorted(date_dict.keys()):
            df = date_dict[date].copy()
            n_samples = len(df)
            
            # 如果標籤已經用完，就直接跳過後面的日期
            if offset >= total_label_len:
                print(f"   ⚠️  {pid}_{date}: 標籤已用完，這一天資料先略過")
                continue
            
            # 從剩餘標籤中取出對應長度
            remaining = total_label_len - offset
            use_len = min(n_samples, remaining)
            
            if use_len <= 0:
                print(f"   ⚠️  {pid}_{date}: 無可用標籤，略過")
                continue
            
            # 如果標籤比資料短，就截斷 df
            if use_len < n_samples:
                print(f"   ⚠️  {pid}_{date}: 資料({n_samples}) > 標籤({use_len})，截斷後面多餘的資料")
                df = df.iloc[:use_len].copy()
                n_samples = use_len
            
            window_labels = labels[offset:offset+n_samples]
            
            df_list.append(df)
            label_list.append(window_labels)
            
            offset += n_samples
        
        if not df_list:
            return None, None
        
        combined_df = pd.concat(df_list, ignore_index=True)
        combined_labels = np.hstack(label_list)
        
        print(f"   → {pid}: 合併後樣本數 = {len(combined_df)}，標籤數 = {len(combined_labels)}")
        
        return combined_df, combined_labels
    
    def _sliding_window_extraction(self, df, labels, pid):
        """滑動窗口提取特徵"""
        features = []
        window_labels = []
        metadata = []
        
        n_samples = len(df)
        step_size = self.window_size  # 不重疊窗口
        
        for start_idx in range(0, n_samples - self.window_size + 1, step_size):
            end_idx = start_idx + self.window_size
            
            window_df = df.iloc[start_idx:end_idx]
            window_label_array = labels[start_idx:end_idx]
            
            # 🛡️ 保險：如果這個窗口沒有任何標籤，就跳過
            if window_label_array.size == 0:
                print(f"   ⚠️  {pid}: [{start_idx}:{end_idx}] 標籤為空，略過此窗口")
                continue
            
            # 該窗口的主要標籤（眾數）
            unique, counts = np.unique(window_label_array, return_counts=True)
            majority_label = unique[np.argmax(counts)]
            
            # 提取特徵
            feature_vector = self._extract_window_features(window_df)
            
            features.append(feature_vector)
            window_labels.append(majority_label)
            metadata.append({
                'pineapple_id': pid,
                'start_idx': start_idx,
                'end_idx': end_idx,
                'majority_label': int(majority_label),
                'label_purity': max(counts) / len(window_label_array)
            })
        
        if len(features) == 0:
            return np.array([]), np.array([]), []
        
        return np.array(features), np.array(window_labels), metadata
    
    def _extract_window_features(self, window_df):
        """從單一窗口提取特徵"""
        features = []
        
        for sensor in self.sensor_cols:
            # 1. Rs/R0 特徵
            col_rs_r0 = f'{sensor}_Rs_R0'
            if col_rs_r0 in window_df.columns:
                data = window_df[col_rs_r0].values
                
                # 統計特徵
                features.append(np.mean(data))           # 平均值
                features.append(np.std(data))            # 標準差
                features.append(np.min(data))            # 最小值
                features.append(np.max(data))            # 最大值
                features.append(np.max(data) - np.min(data))  # 反應幅度
                
                # 變化速率（斜率）
                if len(data) > 1:
                    x = np.arange(len(data))
                    slope, _, _, _, _ = linregress(x, data)
                    features.append(slope)
                else:
                    features.append(0)
                
                # AUC（累積反應）- 修正版
                try:
                    auc = np.trapezoid(data, dx=1)  # numpy >= 2.0
                except AttributeError:
                    # fallback: 手動梯形積分
                    if len(data) > 1:
                        auc = np.sum((data[:-1] + data[1:]) / 2)
                    else:
                        auc = data[0] if len(data) > 0 else 0
                
                features.append(auc)
            else:
                features.extend([0] * 7)  # 填充 7 個特徵
            
            # 2. Delta Rs/R0 特徵
            col_delta = f'{sensor}_delta_Rs_R0'
            if col_delta in window_df.columns:
                data = window_df[col_delta].values
                
                features.append(np.mean(data))
                features.append(np.std(data))
                features.append(np.max(np.abs(data)))    # 最大絕對變化
            else:
                features.extend([0] * 3)
        
        # 3. 跨感測器特徵（比例特徵）
        # MQ3/MQ2 比例（酒精 vs 可燃氣體）
        mq3 = window_df['MQ3_Rs_R0'].mean() if 'MQ3_Rs_R0' in window_df.columns else 0
        mq2 = window_df['MQ2_Rs_R0'].mean() if 'MQ2_Rs_R0' in window_df.columns else 0
        features.append(mq3 / (mq2 + 1e-6))
        
        # MQ135/TGS2602 比例（空氣品質 vs VOC）
        mq135 = window_df['MQ135_Rs_R0'].mean() if 'MQ135_Rs_R0' in window_df.columns else 0
        tgs = window_df['TGS2602_Rs_R0'].mean() if 'TGS2602_Rs_R0' in window_df.columns else 0
        features.append(mq135 / (tgs + 1e-6))
        
        # 所有感測器平均
        all_means = [window_df[f'{s}_Rs_R0'].mean() 
                     for s in self.sensor_cols 
                     if f'{s}_Rs_R0' in window_df.columns]
        features.append(np.mean(all_means) if all_means else 0)
        
        return np.array(features)

    
    def _get_feature_names(self):
        """生成特徵名稱"""
        names = []
        
        for sensor in self.sensor_cols:
            # Rs/R0 特徵
            names.extend([
                f'{sensor}_mean',
                f'{sensor}_std',
                f'{sensor}_min',
                f'{sensor}_max',
                f'{sensor}_range',
                f'{sensor}_slope',
                f'{sensor}_auc'
            ])
            
            # Delta Rs/R0 特徵
            names.extend([
                f'{sensor}_delta_mean',
                f'{sensor}_delta_std',
                f'{sensor}_delta_max_abs'
            ])
        
        # 跨感測器特徵
        names.extend([
            'MQ3_MQ2_ratio',
            'MQ135_TGS2602_ratio',
            'all_sensors_mean'
        ])
        
        return names
    
    def save_features(self):
        """儲存特徵與標籤"""
        print("\n💾 儲存特徵...")
        
        # 1. 特徵矩陣（CSV）
        csv_path = os.path.join(self.output_dir, 'feature_matrix.csv')
        self.feature_matrix.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"   ✅ CSV: {csv_path}")
        
        # 2. 標籤（numpy）
        label_path = os.path.join(self.output_dir, 'labels.npy')
        np.save(label_path, self.labels)
        print(f"   ✅ Labels: {label_path}")
        
        # 3. Metadata（CSV）
        meta_path = os.path.join(self.output_dir, 'metadata.csv')
        self.metadata.to_csv(meta_path, index=False, encoding='utf-8-sig')
        print(f"   ✅ Metadata: {meta_path}")
        
        # 4. 完整 pickle（方便載入）
        pkl_path = os.path.join(self.output_dir, 'feature_data.pkl')
        with open(pkl_path, 'wb') as f:
            pickle.dump({
                'features': self.feature_matrix,
                'labels': self.labels,
                'metadata': self.metadata,
                'feature_names': self.feature_matrix.columns.tolist(),
                'window_size': self.window_size
            }, f)
        print(f"   ✅ Pickle: {pkl_path}")
        
        print("\n✅ Step 4 完成！可以進入 Step 5（模型訓練）")

# 執行特徵工程
feature_engineer = FeatureEngineering(
    arduino_features, 
    maturity_levels,  # 使用重新映射後的 4 等級標籤
    window_size=120   # 120 秒窗口
)

X, y, metadata = feature_engineer.extract_all_features()
feature_engineer.save_features()

# 顯示特徵摘要
print(f"\n📊 特徵摘要:")
print(f"   總樣本數: {len(y)}")
print(f"   特徵維度: {X.shape[1]}")
print(f"   特徵列表: {X.columns.tolist()[:10]}... (前10個)")


🔧 Step 4: 特徵工程（固定時間窗）
窗口大小: 120 秒

🍍 處理 Pineapple 01...
   → 01: 合併後樣本數 = 7200，標籤數 = 7200
   ✓ 提取 60 個窗口，每個 53 維特徵

🍍 處理 Pineapple 02...
   → 02: 合併後樣本數 = 6300，標籤數 = 6300
   ✓ 提取 52 個窗口，每個 53 維特徵

🍍 處理 Pineapple 03...
   → 03: 合併後樣本數 = 6300，標籤數 = 6300
   ✓ 提取 52 個窗口，每個 53 維特徵

🍍 處理 Pineapple 04...
   → 04: 合併後樣本數 = 6299，標籤數 = 6299
   ✓ 提取 52 個窗口，每個 53 維特徵

🍍 處理 Pineapple 05...
   → 05: 合併後樣本數 = 3600，標籤數 = 3600
   ✓ 提取 30 個窗口，每個 53 維特徵

🍍 處理 Pineapple 06...
   → 06: 合併後樣本數 = 3599，標籤數 = 3599
   ✓ 提取 29 個窗口，每個 53 維特徵

🍍 處理 Pineapple 07...
   → 07: 合併後樣本數 = 6298，標籤數 = 6298
   ✓ 提取 52 個窗口，每個 53 維特徵

🍍 處理 Pineapple 08...
   → 08: 合併後樣本數 = 6300，標籤數 = 6300
   ✓ 提取 52 個窗口，每個 53 維特徵

🍍 處理 Pineapple 09...
   → 09: 合併後樣本數 = 1800，標籤數 = 1800
   ✓ 提取 15 個窗口，每個 53 維特徵

🍍 處理 Pineapple 10...
   → 10: 合併後樣本數 = 1800，標籤數 = 1800
   ✓ 提取 15 個窗口，每個 53 維特徵

🍍 處理 Pineapple 11...
   → 11: 合併後樣本數 = 1800，標籤數 = 1800
   ✓ 提取 15 個窗口，每個 53 維特徵

✅ 特徵提取完成！
   特徵矩陣: (424, 53)
   標籤數量: 424
   標籤分布: {np.int64(0): np.int64(